# Run Delphes and extract observables

Johann Brehmer, Kyle Cranmer, Felix Kling, Duccio Pappadopulo, Josh Ruderman 2018

In [2]:
from __future__ import absolute_import, division, print_function, unicode_literals

import numpy as np
import matplotlib
from matplotlib import pyplot as plt
% matplotlib inline
import logging
import os

from madminer.delphes import DelphesProcessor
from madminer.sampling import combine_and_shuffle

logging.basicConfig(format='%(asctime)s  %(message)s', datefmt='%H:%M')


In [3]:
base_dir = '/Users/johannbrehmer/work/projects/madminer/diboson_mining/'
mg_dir = '/Users/johannbrehmer/work/projects/madminer/MG5_aMC_v2_6_2/'

In [4]:
sample_dir = base_dir + 'data/samples/wgamma/'
card_dir = base_dir + 'cards/wgamma/'
ufo_model_dir = card_dir + 'SMWgamma_UFO'
run_card_dir = card_dir + 'run_cards/'
mg_process_dir = base_dir + 'data/mg_processes/wgamma/'
log_dir = base_dir + 'logs/wgamma/'
temp_dir = base_dir + 'data/temp'
delphes_dir = mg_dir + 'Delphes'

## Define observables

In [6]:
# Number of particles considered
n_leptons = 1
n_photons = 2
n_jets = 1

def setup_observables(delphesprocessor):
    # ETMiss
    delphesprocessor.add_default_observables(
        n_leptons_max=n_leptons,
        n_photons_max=n_photons,
        n_jets_max=n_jets,
        include_met=True
    )
        
    # Correlations with MET
    for s1, i1 in [('l', 0), ('a', 0)]:
        delphesprocessor.add_observable(
            'deltaphi_{}{}_met'.format(s1, i1+1),
            'abs({}[{}].phi() - met.phi())'.format(s1, i1),
            required=False,
            default=0.
        )

    # Selected correlations between particles
    for s1, i1, s2, i2 in [('l', 0, 'a', 0)]:
        delphesprocessor.add_observable(
            'm_{}{}_{}{}'.format(s1, i1+1, s2, i2+1),
            '({}[{}] + {}[{}]).m'.format(s1, i1, s2, i2),
            required=False,
            default=0.
        )
        delphesprocessor.add_observable(
            'deltaeta_{}{}_{}{}'.format(s1, i1+1, s2, i2+1),
            'abs({}[{}].eta - {}[{}].eta)'.format(s1, i1, s2, i2),
            required=False,
            default=0.
        )
        delphesprocessor.add_observable(
            'deltaphi_{}{}_{}{}'.format(s1, i1+1, s2, i2+1),
            'abs({}[{}].phi() - {}[{}].phi())'.format(s1, i1, s2, i2),
            required=False,
            default=0.
        )


In [7]:
n_runs_per_benchmark = 18  # Number of run_cards

In [5]:
run = 0

for i_card in range(n_runs_per_benchmark):
    logging.info('Starting analysis of runs for card {}'.format(i_card))
            
    # Load setup
    dp = DelphesProcessor(sample_dir + 'setup.h5')
    
    # Load events
    for benchmark in dp.benchmark_names:
        run += 1
        run_str = str(run)
        if len(run_str) < 2:
            run_str = '0' + run_str

        dp.add_hepmc_sample(
            mg_process_dir + 'Events/run_{}/tag_1_pythia8_events.hepmc.gz'.format(run_str),
            sampled_from_benchmark=benchmark
        )
        
    # Run Delphes
    dp.run_delphes(
        delphes_directory=delphes_dir,
        delphes_card=card_dir + 'delphes_card.dat',
        log_directory=log_dir,
        initial_command='source activate python2'
    )
    
    # Set up observables
    setup_observables(dp)
    
    # Analyse Delphes sample
    dp.analyse_delphes_samples()
        
    # Save
    dp.save(sample_dir + 'samples_{}.h5'.format(i_card))
    
    # Clean up
    os.remove(mg_process_dir + 'Events/run_{}/tag_1_pythia8_events.hepmc'.format(run_str))
    os.remove(mg_process_dir + 'Events/run_{}/tag_1_pythia8_events_delphes.root'.format(run_str))


11:51  
11:51  ------------------------------------------------------------
11:51  |                                                          |
11:51  |  MadMiner v2018.10.10                                    |
11:51  |                                                          |
11:51  |           Johann Brehmer, Kyle Cranmer, and Felix Kling  |
11:51  |                                                          |
11:51  ------------------------------------------------------------
11:51  
11:51  Adding HepMC sample at /Users/johannbrehmer/work/projects/madminer/diboson_mining/data/mg_processes/wgamma/Events/run_01/tag_1_pythia8_events.hepmc.gz
11:51  Adding HepMC sample at /Users/johannbrehmer/work/projects/madminer/diboson_mining/data/mg_processes/wgamma/Events/run_02/tag_1_pythia8_events.hepmc.gz
11:51  Adding HepMC sample at /Users/johannbrehmer/work/projects/madminer/diboson_mining/data/mg_processes/wgamma/Events/run_03/tag_1_pythia8_events.hepmc.gz
11:51  Adding HepMC sample at /Use

15:51  Running Delphes (/Users/johannbrehmer/work/projects/madminer/MG5_aMC_v2_6_2/Delphes) on event sample at /Users/johannbrehmer/work/projects/madminer/diboson_mining/data/mg_processes/wgamma/Events/run_08/tag_1_pythia8_events.hepmc.gz
15:51  Running Delphes (/Users/johannbrehmer/work/projects/madminer/MG5_aMC_v2_6_2/Delphes) on event sample at /Users/johannbrehmer/work/projects/madminer/diboson_mining/data/mg_processes/wgamma/Events/run_09/tag_1_pythia8_events.hepmc.gz
15:51  Running Delphes (/Users/johannbrehmer/work/projects/madminer/MG5_aMC_v2_6_2/Delphes) on event sample at /Users/johannbrehmer/work/projects/madminer/diboson_mining/data/mg_processes/wgamma/Events/run_10/tag_1_pythia8_events.hepmc.gz
15:52  Running Delphes (/Users/johannbrehmer/work/projects/madminer/MG5_aMC_v2_6_2/Delphes) on event sample at /Users/johannbrehmer/work/projects/madminer/diboson_mining/data/mg_processes/wgamma/Events/run_11/tag_1_pythia8_events.hepmc.gz
15:52  Running Delphes (/Users/johannbrehmer

15:54  Requiring existence of observable n_ls: 10000 events pass, 0 events removed
15:54  Requiring existence of observable n_as: 10000 events pass, 0 events removed
15:54  Requiring existence of observable n_js: 10000 events pass, 0 events removed
15:55  Analysing Delphes sample /Users/johannbrehmer/work/projects/madminer/diboson_mining/data/mg_processes/wgamma/Events/run_14/tag_1_pythia8_events_delphes.root
15:55  Requiring existence of observable et_miss: 10000 events pass, 0 events removed
15:55  Requiring existence of observable phi_miss: 10000 events pass, 0 events removed
15:55  Requiring existence of observable n_ls: 10000 events pass, 0 events removed
15:55  Requiring existence of observable n_as: 10000 events pass, 0 events removed
15:55  Requiring existence of observable n_js: 10000 events pass, 0 events removed
15:55  Analysing Delphes sample /Users/johannbrehmer/work/projects/madminer/diboson_mining/data/mg_processes/wgamma/Events/run_15/tag_1_pythia8_events_delphes.root
1

15:57  Requiring existence of observable et_miss: 10000 events pass, 0 events removed
15:57  Requiring existence of observable phi_miss: 10000 events pass, 0 events removed
15:57  Requiring existence of observable n_ls: 10000 events pass, 0 events removed
15:57  Requiring existence of observable n_as: 10000 events pass, 0 events removed
15:57  Requiring existence of observable n_js: 10000 events pass, 0 events removed
15:57  Analysing Delphes sample /Users/johannbrehmer/work/projects/madminer/diboson_mining/data/mg_processes/wgamma/Events/run_23/tag_1_pythia8_events_delphes.root
15:57  Requiring existence of observable et_miss: 10000 events pass, 0 events removed
15:57  Requiring existence of observable phi_miss: 10000 events pass, 0 events removed
15:57  Requiring existence of observable n_ls: 10000 events pass, 0 events removed
15:57  Requiring existence of observable n_as: 10000 events pass, 0 events removed
15:57  Requiring existence of observable n_js: 10000 events pass, 0 events 

17:18  Loading HDF5 data from /Users/johannbrehmer/work/projects/madminer/diboson_mining/data/samples/wgamma/setup.h5 and saving file to /Users/johannbrehmer/work/projects/madminer/diboson_mining/data/samples/wgamma/samples_4.h5
17:18  Starting analysis of runs for card 5
17:18  
17:18  ------------------------------------------------------------
17:18  |                                                          |
17:18  |  MadMiner v2018.10.10                                    |
17:18  |                                                          |
17:18  |           Johann Brehmer, Kyle Cranmer, and Felix Kling  |
17:18  |                                                          |
17:18  ------------------------------------------------------------
17:18  
17:18  Adding HepMC sample at /Users/johannbrehmer/work/projects/madminer/diboson_mining/data/mg_processes/wgamma/Events/run_31/tag_1_pythia8_events.hepmc.gz
17:18  Adding HepMC sample at /Users/johannbrehmer/work/projects/madminer/dib

...

17:38  Running Delphes (/Users/johannbrehmer/work/projects/madminer/MG5_aMC_v2_6_2/Delphes) on event sample at /Users/johannbrehmer/work/projects/madminer/diboson_mining/data/mg_processes/wgamma/Events/run_78/tag_1_pythia8_events.hepmc.gz
17:38  Analysing Delphes sample /Users/johannbrehmer/work/projects/madminer/diboson_mining/data/mg_processes/wgamma/Events/run_73/tag_1_pythia8_events_delphes.root
17:38  Requiring existence of observable et_miss: 10000 events pass, 0 events removed
17:38  Requiring existence of observable phi_miss: 10000 events pass, 0 events removed
17:38  Requiring existence of observable n_ls: 10000 events pass, 0 events removed
17:38  Requiring existence of observable n_as: 10000 events pass, 0 events removed
17:38  Requiring existence of observable n_js: 10000 events pass, 0 events removed
17:38  Analysing Delphes sample /Users/johannbrehmer/work/projects/madminer/diboson_mining/data/mg_processes/wgamma/Events/run_74/tag_1_pythia8_events_delphes.root
17:38  Requ

17:41  Requiring existence of observable et_miss: 10000 events pass, 0 events removed
17:41  Requiring existence of observable phi_miss: 10000 events pass, 0 events removed
17:41  Requiring existence of observable n_ls: 10000 events pass, 0 events removed
17:41  Requiring existence of observable n_as: 10000 events pass, 0 events removed
17:41  Requiring existence of observable n_js: 10000 events pass, 0 events removed
17:41  Analysing Delphes sample /Users/johannbrehmer/work/projects/madminer/diboson_mining/data/mg_processes/wgamma/Events/run_82/tag_1_pythia8_events_delphes.root
17:41  Requiring existence of observable et_miss: 10000 events pass, 0 events removed
17:41  Requiring existence of observable phi_miss: 10000 events pass, 0 events removed
17:41  Requiring existence of observable n_ls: 10000 events pass, 0 events removed
17:41  Requiring existence of observable n_as: 10000 events pass, 0 events removed
17:41  Requiring existence of observable n_js: 10000 events pass, 0 events 

17:44  Analysing Delphes sample /Users/johannbrehmer/work/projects/madminer/diboson_mining/data/mg_processes/wgamma/Events/run_90/tag_1_pythia8_events_delphes.root
17:44  Requiring existence of observable et_miss: 10000 events pass, 0 events removed
17:44  Requiring existence of observable phi_miss: 10000 events pass, 0 events removed
17:44  Requiring existence of observable n_ls: 10000 events pass, 0 events removed
17:44  Requiring existence of observable n_as: 10000 events pass, 0 events removed
17:44  Requiring existence of observable n_js: 10000 events pass, 0 events removed
17:44  Loading HDF5 data from /Users/johannbrehmer/work/projects/madminer/diboson_mining/data/samples/wgamma/setup.h5 and saving file to /Users/johannbrehmer/work/projects/madminer/diboson_mining/data/samples/wgamma/samples_14.h5
17:44  Starting analysis of runs for card 15
17:44  
17:44  ------------------------------------------------------------
17:44  |                                                       

17:47  Adding HepMC sample at /Users/johannbrehmer/work/projects/madminer/diboson_mining/data/mg_processes/wgamma/Events/run_99/tag_1_pythia8_events.hepmc.gz
17:47  Adding HepMC sample at /Users/johannbrehmer/work/projects/madminer/diboson_mining/data/mg_processes/wgamma/Events/run_100/tag_1_pythia8_events.hepmc.gz
17:47  Adding HepMC sample at /Users/johannbrehmer/work/projects/madminer/diboson_mining/data/mg_processes/wgamma/Events/run_101/tag_1_pythia8_events.hepmc.gz
17:47  Adding HepMC sample at /Users/johannbrehmer/work/projects/madminer/diboson_mining/data/mg_processes/wgamma/Events/run_102/tag_1_pythia8_events.hepmc.gz
17:47  Running Delphes (/Users/johannbrehmer/work/projects/madminer/MG5_aMC_v2_6_2/Delphes) on event sample at /Users/johannbrehmer/work/projects/madminer/diboson_mining/data/mg_processes/wgamma/Events/run_97/tag_1_pythia8_events.hepmc.gz
17:48  Running Delphes (/Users/johannbrehmer/work/projects/madminer/MG5_aMC_v2_6_2/Delphes) on event sample at /Users/johannbr

17:52  Running Delphes (/Users/johannbrehmer/work/projects/madminer/MG5_aMC_v2_6_2/Delphes) on event sample at /Users/johannbrehmer/work/projects/madminer/diboson_mining/data/mg_processes/wgamma/Events/run_107/tag_1_pythia8_events.hepmc.gz
17:52  Running Delphes (/Users/johannbrehmer/work/projects/madminer/MG5_aMC_v2_6_2/Delphes) on event sample at /Users/johannbrehmer/work/projects/madminer/diboson_mining/data/mg_processes/wgamma/Events/run_108/tag_1_pythia8_events.hepmc.gz
17:53  Analysing Delphes sample /Users/johannbrehmer/work/projects/madminer/diboson_mining/data/mg_processes/wgamma/Events/run_103/tag_1_pythia8_events_delphes.root
17:53  Requiring existence of observable et_miss: 10000 events pass, 0 events removed
17:53  Requiring existence of observable phi_miss: 10000 events pass, 0 events removed
17:53  Requiring existence of observable n_ls: 10000 events pass, 0 events removed
17:53  Requiring existence of observable n_as: 10000 events pass, 0 events removed
17:53  Requiring

## Main loop

## Combine samples

In [8]:
filenames_in = [sample_dir + 'samples_{}.h5'.format(i_card) for i_card in range(n_runs_per_benchmark)]

combine_and_shuffle(filenames_in, sample_dir + 'samples.h5')

09:34  
09:34  ------------------------------------------------------------
09:34  |                                                          |
09:34  |  MadMiner v2018.10.10                                    |
09:34  |                                                          |
09:34  |           Johann Brehmer, Kyle Cranmer, and Felix Kling  |
09:34  |                                                          |
09:34  ------------------------------------------------------------
09:34  
09:34  Careful: this tool assumes that all samples are generated with the same setup, including identical benchmarks (and thus morphing setup). If it is used with samples with different settings, there will be wrong results! There are no explicit cross checks in place yet.
09:34  Copying setup from /Users/johannbrehmer/work/projects/madminer/diboson_mining/data/samples/wgamma/samples_0.h5 to /Users/johannbrehmer/work/projects/madminer/diboson_mining/data/samples/wgamma/samples.h5
09:34  Loading samples 